In [1]:
import pickle
import h5py
import os
import torch
import random
import torch.utils.data as data
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import time

/home/swjung/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Set HyperParameter
learning_rate = 0.00001
batch_size = 50
nb_classes = 2
nb_epoch = 5
img_rows = 40
img_cols = 101
nb_filters = 100
nb_pool = 2
nb_conv = 3
epochs = 50

test_range = 32
total_process_count = test_range * epochs

TEST_TYPE = 0

if TEST_TYPE == 0:
    _type = 'Valence'
elif TEST_TYPE == 1:
    _type = 'Arousal'
else:
    raise ValueError('TEST_TYPE must be 1 or 2')

In [3]:
class DEAP_DataSet(data.Dataset):
    
    VALID_SPLIT = ('train', 'validation', 'test')
    DATASET = 'data/'
    
    def __init__(self, test_target, test_type=0, split='train', data_dir='processed_data.pkl', label_dir='y_train.pkl'):
        super(DEAP_DataSet, self).__init__()
        
        if split not in self.VALID_SPLIT:
            raise ValueError('Unknown split {:s}'.format(split))
        if not os.path.exists(self.DATASET + data_dir):
            raise ValueError('{:s} does not exist'.format(data_dir))
        if not os.path.exists(self.DATASET + label_dir):
            raise ValueError('{:s} does not exist'.format(label_dir))
            
        data_path = '{}/{}'.format(self.DATASET, data_dir)
        label_path = '{}/{}'.format(self.DATASET, label_dir)
                
        self.split = split
        self.data = pickle.load(open(data_path, 'rb'))
        self.labels = pickle.load(open(label_path, 'rb'))
        
        self.test_target = test_target
        self.test_type = test_type
        self.target_data = []
        
        # create train data set
        if self.split == self.VALID_SPLIT[0]:
            counter = 0
            for pp, pp_data in enumerate(self.data):
                if self.test_target is not pp:
                    for c, data in enumerate(pp_data):
                        counter += 1
                        temp_label = self.labels[pp][c][self.test_type]
                        temp_data = np.zeros((1, 40, 101))
                        temp_data[0, :, :] = 1
                        
                        for ch in range(40):
                            for d in range(101):
                                temp_data[0][ch][d] = data[ch][d]

                        self.target_data.append((temp_label, temp_data))
            
        # craete test data set
        if self.split == self.VALID_SPLIT[2]:
            counter = 0
            for pp, pp_data in enumerate(self.data):
                if self.test_target == pp:
                    for c, data in enumerate(pp_data):
                        counter += 1
                        temp_label = self.labels[pp][c][self.test_type]
                        temp_data = np.zeros((1, 40, 101))
                        temp_data[0, :, :] = 1
                        
                        for ch in range(40):
                            for d in range(101):
                                temp_data[0][ch][d] = data[ch][d]
                        self.target_data.append((temp_label, temp_data))
            
    def __getitem__(self, index):
        return self.target_data[index]
    
    def __len__(self):
        return len(self.target_data)
    
    def collate_fn(self, batch):        
        label = [b[0] for b in batch]
        # (C, D, H, W): (1, 40, 101)
        data = [b[1] for b in batch]
        data = [torch.FloatTensor(v).unsqueeze(0) for v in data]
        return torch.LongTensor(label), torch.cat(data, dim=0)

In [4]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.cnn_layer = nn.Sequential(
            nn.Conv2d(1, nb_filters, (nb_conv, nb_conv)),
            nn.Tanh(),
            nn.Conv2d(nb_filters, nb_filters, (nb_conv, nb_conv)),
            nn.Tanh(),
            nn.MaxPool2d(nb_pool, nb_pool),
            nn.Dropout2d(0.25),
        )
        self.fully_layer = nn.Sequential(
            nn.Linear(86400, 100),
            nn.Tanh(),
            nn.Dropout(0.25),
            nn.Linear(100, 2),
            nn.Softplus()
        )
        
    def forward(self, x):
        out = self.cnn_layer(x)
        out = out.view(x.size(0), -1)
        out = self.fully_layer(out)
        
        return out
        

In [21]:
class CNN2(nn.Module):
    def __init__(self):
        super(CNN2, self).__init__()
        self.cnn_layer = nn.Sequential(
            nn.Conv2d(1, nb_filters, (nb_conv, nb_conv)),
            nn.ReLU(),
            nn.Conv2d(nb_filters, nb_filters, (nb_conv, nb_conv)),
            nn.Tanh(),
            nn.MaxPool2d(nb_pool, nb_pool),
            nn.Dropout2d(0.25),
        )
        self.fully_layer = nn.Sequential(
            nn.Linear(86400, 100),
            nn.Tanh(),
            nn.Dropout(0.25),
            nn.Linear(100, 2),
            nn.Softplus()
        )
        
    def forward(self, x):
        out = self.cnn_layer(x)
        out = out.view(x.size(0), -1)
        out = self.fully_layer(out)
        
        return out
        

In [6]:
def run_train(test_num, test_type):
    train_model = CNN().double().cuda()
    
    DEAP_train = DEAP_DataSet(test_num, test_type=test_type, split='train')
    train_loader = torch.utils.data.DataLoader(DEAP_train, batch_size=batch_size, shuffle=False, num_workers=2, drop_last=False)
    
    optimizer = torch.optim.SGD(train_model.parameters(), lr=learning_rate, momentum=0.9)
    loss_func = nn.CrossEntropyLoss().double()
    
    for epoch in range(epochs):
        count = 0
        for label, data in train_loader:
            x = Variable(data).double().cuda()
            y_ = Variable(label).type(torch.LongTensor).cuda()

            optimizer.zero_grad()
            output = train_model.forward(x)
            loss = loss_func(output, y_).double()
            loss.backward()
            optimizer.step()

            count += 1
    return train_model

In [7]:
def run_validation(model, test_num, test_type):
    DEAP_test = DEAP_DataSet(test_num, test_type=test_type, split='test')
    test_loader = torch.utils.data.DataLoader(DEAP_test, batch_size=batch_size, shuffle=False, num_workers=2, drop_last=False)
    
    total = 0
    correct = 0
    
    for label, data in test_loader:
        x = Variable(data, requires_grad=False).double().cuda()
        y_ = Variable(label).type(torch.LongTensor).cuda()
        
        with torch.no_grad():
            output = model.forward(x)
            _, output_index = torch.max(output, 1)

            total += label.size(0)
            correct += (output_index == y_).sum().float()
    return total, correct

In [8]:
def get_min_sec(t):
    _min = int(t / 60)
    _sec = int(t) % 60
    
    return _min, _sec

In [10]:
total = 0
correct = 0
for test_num in range(test_range):
    
    start_time = time.time()
    print('# > [{}/{}] train start.'.format(test_num + 1, test_range))
    model = run_train(test_num, TEST_TYPE)
    print('# > [{}/{}] train end.'.format(test_num + 1, test_range))
    
    temp_total = 0
    temp_correct = 0
    for c in range(30):
        sub_total, sub_correct = run_validation(model, test_num, TEST_TYPE)
        temp_total += sub_total
        temp_correct += sub_correct
        
    current_process_count = (test_num + 1) * epochs
    end_time = time.time()
    run_time = (end_time - start_time)
    remain_process = test_range - 1 - test_num
    remain_time = run_time * remain_process
    
    _min, _sec = get_min_sec(remain_time)
    
    print('# > [{}/{}] correct : {}% ({} / {}) -> {}%, remain time : {}mins {}secs'.format(test_num + 1, test_range, 100 * temp_correct / temp_total, temp_correct, temp_total, 100 * current_process_count / total_process_count, _min, _sec))
    
    total += temp_total
    correct += temp_correct
    
    print('# > total :  correct : {}% ({} / {})'.format(100 * correct / total, correct, total))

# > [1/32] train start.
# > [1/32] train end.
# > [1/32] correct : 49.16666793823242% (590.0 / 1200) -> 3.125%, remain time : 246mins 5secs
# > total :  correct : 49.16666793823242% (590.0 / 1200)
# > [2/32] train start.
# > [2/32] train end.
# > [2/32] correct : 63.41666793823242% (761.0 / 1200) -> 6.25%, remain time : 237mins 12secs
# > total :  correct : 56.29166793823242% (1351.0 / 2400)
# > [3/32] train start.
# > [3/32] train end.
# > [3/32] correct : 57.083335876464844% (685.0 / 1200) -> 9.375%, remain time : 229mins 14secs
# > total :  correct : 56.55555725097656% (2036.0 / 3600)
# > [4/32] train start.
# > [4/32] train end.
# > [4/32] correct : 40.66666793823242% (488.0 / 1200) -> 12.5%, remain time : 221mins 26secs
# > total :  correct : 52.583335876464844% (2524.0 / 4800)
# > [5/32] train start.
# > [5/32] train end.
# > [5/32] correct : 62.583335876464844% (751.0 / 1200) -> 15.625%, remain time : 213mins 36secs
# > total :  correct : 54.58333206176758% (3275.0 / 6000)
# > [

In [11]:
print('# > train and validation end')
print('# > final report')
print('type : {}, result : {}% ({} / {})'.format(_type, 100 * correct / total, correct, total))

# > train and validation end
# > final report
type : Valence, result : 57.265625% (21990.0 / 38400)


In [22]:

TEST_TYPE = 1
learning_rate = 0.001

if TEST_TYPE == 0:
    _type = 'Valence'
elif TEST_TYPE == 1:
    _type = 'Arousal'
else:
    raise ValueError('TEST_TYPE must be 1 or 2')

In [23]:
def run_train(test_num, test_type):
    train_model = CNN2().double().cuda()
    
    DEAP_train = DEAP_DataSet(test_num, test_type=test_type, split='train')
    train_loader = torch.utils.data.DataLoader(DEAP_train, batch_size=batch_size, shuffle=False, num_workers=2, drop_last=False)
    
    optimizer = torch.optim.SGD(train_model.parameters(), lr=learning_rate, momentum=0.9)
    loss_func = nn.CrossEntropyLoss().double()
    
    for epoch in range(epochs):
        count = 0
        for label, data in train_loader:
            x = Variable(data).double().cuda()
            y_ = Variable(label).type(torch.LongTensor).cuda()

            optimizer.zero_grad()
            output = train_model.forward(x)
            loss = loss_func(output, y_).double()
            loss.backward()
            optimizer.step()

            count += 1
    return train_model

In [ ]:
total = 0
correct = 0
for test_num in range(test_range):
    
    start_time = time.time()
    print('# > [{}/{}] train start.'.format(test_num + 1, test_range))
    model = run_train(test_num, TEST_TYPE)
    print('# > [{}/{}] train end.'.format(test_num + 1, test_range))
    
    temp_total = 0
    temp_correct = 0
    for c in range(30):
        sub_total, sub_correct = run_validation(model, test_num, TEST_TYPE)
        temp_total += sub_total
        temp_correct += sub_correct
        
    current_process_count = (test_num + 1) * epochs
    end_time = time.time()
    run_time = (end_time - start_time)
    remain_process = test_range - 1 - test_num
    remain_time = run_time * remain_process
    
    _min, _sec = get_min_sec(remain_time)
    
    print('# > [{}/{}] correct : {}% ({} / {}) -> {}%, remain time : {}mins {}secs'.format(test_num + 1, test_range, 100 * temp_correct / temp_total, temp_correct, temp_total, 100 * current_process_count / total_process_count, _min, _sec))
    
    total += temp_total
    correct += temp_correct
    print('# > total :  correct : {}% ({} / {})'.format(100 * correct / total, correct, total))

# > [1/32] train start.
# > [1/32] train end.
# > [1/32] correct : 60.0% (720.0 / 1200) -> 3.125%, remain time : 241mins 13secs
# > total :  correct : 60.0% (720.0 / 1200)
# > [2/32] train start.
# > [2/32] train end.
# > [2/32] correct : 60.083335876464844% (721.0 / 1200) -> 6.25%, remain time : 235mins 12secs
# > total :  correct : 60.04166793823242% (1441.0 / 2400)
# > [3/32] train start.
# > [3/32] train end.
# > [3/32] correct : 20.0% (240.0 / 1200) -> 9.375%, remain time : 227mins 11secs
# > total :  correct : 46.6944465637207% (1681.0 / 3600)
# > [4/32] train start.
# > [4/32] train end.
# > [4/32] correct : 40.5% (486.0 / 1200) -> 12.5%, remain time : 219mins 15secs
# > total :  correct : 45.145835876464844% (2167.0 / 4800)
# > [5/32] train start.
# > [5/32] train end.
# > [5/32] correct : 47.16666793823242% (566.0 / 1200) -> 15.625%, remain time : 211mins 24secs
# > total :  correct : 45.54999923706055% (2733.0 / 6000)
# > [6/32] train start.
# > [6/32] train end.
# > [6/32] c

In [ ]:
print('# > train and validation end')
print('# > final report')
print('type : {}, result : {}% ({} / {})'.format(_type, 100 * correct / total, correct, total))